In [14]:
const start_pos = 37 #start position (bottom-left)
const goal_pos = 48 #goal state (bottom-right)
Q = zeros(48, 4) #intialize q values

function nextstate(s, a)
    rows, cols = 4, 12 #grid dims
    row, col = div(s - 1, cols) + 1, (s - 1) % cols + 1  #converts our state to row,col

    if a == 1  # Up
        row = max(row - 1, 1)  #stay on the grid
    elseif a == 2  # Down
        row = min(row + 1, rows) #stay on the grid
    elseif a == 3  # Right
        col = min(col + 1, cols) #stay on the grid
    elseif a == 4  # Left
        col = max(col - 1, 1) #stay on the grid
    end
    if row == 4 && 2 <= col <= 11  #see if the agent falls of the cliff (4,2 to 4,11)
        return start_pos  #go back to the start if he falls
    end
    return (row - 1) * cols + col #convert back into a state
end

function epsilon_greedy_policy(pos, Q)
    if rand() < 0.1
        return rand(1:NUM_ACTIONS)  #choose a random action
    else
        return argmax(Q[pos, :])  #choose a greedy action
    end
end

function q_learning(episodes)
    for episode in 1:episodes #loop over the episode number specified
        pos = start_pos #set the position of the agent
        while pos != goal_pos #loop while we are not at the end goal position
            action = epsilon_greedy_policy(pos, Q) #select your action
            next_state = nextstate(pos, action) #find your position based on your action
            Q[pos, action] += 0.1 * (-1 + 0.9 * maximum(Q[next_state, :]) - Q[pos, action]) #update the q matrix
            pos = next_state #update your position
        end
    end
end

q_learning(10000) #run the q learning with 10000 episodes
println("Learned policy (1 = up, 2 = down, 3 = right, 4 = left):") #print out a legend
policy = [argmax(Q[s, :]) for s in 1:48] #find the best action for each position by taking the max q value at that index
policy_grid = reshape(policy, (12, 4))' #reshape the grid to visualize (i transposed it because it was originally showing as a 12x4 grid)
for row in 1:size(policy_grid, 1)
    println(policy_grid[row, :]) #print out the optimal policy
end

Learned policy (1 = up, 2 = down, 3 = right, 4 = left):
[3, 3, 3, 3, 1, 3, 2, 2, 3, 3, 2, 2]
[2, 3, 2, 3, 3, 2, 2, 3, 2, 2, 2, 2]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


As you can see above, our policy grid is
[3, 3, 3, 3, 1, 3, 2, 2, 3, 3, 2, 2]
[2, 3, 2, 3, 3, 2, 2, 3, 2, 2, 2, 2]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
The bottom left corner is where we start, and we want to get to the bottom right, avoiding a cliff that spans for the rest of the bottom row (column 2-11). It matches the policy in the book, as we move up, then right until we are unable to, and then down to reach the destination. the values in the first 2 rows are arbitrarily set at the beginning and do not involve our optimal path, so they will be random upon each generation of this problem.